In [182]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
from spotipy.oauth2 import SpotifyOAuth
from tqdm import tqdm

# Credential

In [69]:
credentials = json.load(open('authorization.json'))
client_id = credentials['client_id']
client_secret = credentials['client_secret']
client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=6, retries=5)

# Extract general genres names

In [152]:
genres = []
categories = sp.categories(limit=50, offset=0)['categories']['items']
for cat in categories:
    genres.append((cat['id'],cat['name']))

Filter unuseful genres



Pop,Dance/Electronic, Hip-Hop,Rock,Indie,R&B,Jazz,Classical,Alternative,Instrumental,Metal,Blues,Folk & Acoustic, K-Pop,Mandopop,Malay Pop,Latin,Cantopop,Musik Indonesia

In [153]:
valid_genres = [
 'Pop',
 'Dance/Electronic',
 'Hip-Hop',
 'K-Pop',
 'Mandopop',
 'Rock',
 'Indie',
 'R&B',
 'Jazz',
 'Classical',
 'Alternative',
 'Instrumental',
 'Malay Pop',
 'Latin',
 'Cantopop',
 'Musik Indonesia',
 'Metal',
 'Blues',
 'Folk & Acoustic']
genres = [[id,name] for id, name in genres if name in valid_genres]

In [155]:
genres = pd.DataFrame(genres, columns=['Genre_id','Genre_name'])

# Get every genre playlists

In [175]:
pls = pd.DataFrame(columns = ['Genre_id','Genre_name',"id"])
for i in range(len(genres)):
    genre = genres.loc[i,'Genre_id']
    playlists = []
    try:
        pl = sp.category_playlists(category_id=genre, limit=50, offset=0)["playlists"]
        total = int(pl["total"])
        playlists =  pl['items']
        if total > 50:
            for j in range(50,total,50):
                playlists =  playlists + sp.category_playlists(category_id=genre, limit=50, offset=j)["playlists"]['items']
    except:
        print(genre)
        client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)
        sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=6, retries=5)
    finally:
        pl = sp.category_playlists(category_id=genre, limit=50, offset=0)["playlists"]
        total = int(pl["total"])
        playlists =  pl['items']
        if total > 50:
            for j in range(50,total,50):
                playlists =  playlists + sp.category_playlists(category_id=genre, limit=50, offset=j)["playlists"]['items']
  

    playlists = pd.json_normalize(playlists)
    playlists['Genre_name'] = genres.loc[i,'Genre_name']
    playlists['Genre_id'] = genres.loc[i,'Genre_id']
    pls = pd.concat([pls,playlists[['Genre_id','Genre_name',"id"]]], axis=0)
    

In [177]:
pls.columns = ['Genre_id','Genre_name','playlist_id']
pls = pls[pd.notnull(pls.playlist_id)]
pls.reset_index(inplace=True)

# Extract tracks of each playlist

In [209]:
tracks_total = []   
for i in tqdm(range(len(pls))):
    t = pls.iloc[i]
    try:
        tracks = sp.playlist_items(t['playlist_id'], limit=50, offset=0)['items']
        total = sp.playlist_items(t['playlist_id'], limit=50, offset=0)['total']
        for track in tracks:
            if track['track'] != None:
                tracks_total.append([t['Genre_id'],t['Genre_name'],t['playlist_id'],track['track']['id']])
        if total>50:
            for j in range(50,total,50):
                tracks = sp.playlist_items(t['playlist_id'], limit=50, offset=j)['items']
                for track in tracks:
                    if track['track'] != None:
                        tracks_total.append([t['Genre_id'],t['Genre_name'],t['playlist_id'],track['track']['id']])
    except:
        print(t)
        client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)
        sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=6, retries=5)
    finally:
        tracks = sp.playlist_items(t['playlist_id'], limit=50, offset=0)['items']
        total = sp.playlist_items(t['playlist_id'], limit=50, offset=0)['total']
        for track in tracks:
            if track['track'] != None:
                tracks_total.append([t['Genre_id'],t['Genre_name'],t['playlist_id'],track['track']['id']])
        if total>50:
            for j in range(50,total,50):
                tracks = sp.playlist_items(t['playlist_id'], limit=50, offset=j)['items']
                for track in tracks:
                    if track['track'] != None:
                        tracks_total.append([t['Genre_id'],t['Genre_name'],t['playlist_id'],track['track']['id']])


100%|██████████| 4/4 [00:07<00:00,  1.99s/it]


In [210]:
tracks_total = pd.DataFrame(tracks_total, columns=['Genre_id','Genre_name','playlist_id','track_id'])

In [211]:
tracks_total.to_csv("tracks_total_" + "4" + ".csv")